In [1]:
import pandas as pd
import re
import requests
import hashlib
import json
import openai

/Users/gengliu/anaconda3/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (4.0.0)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn(


### find the unresponded queries from google and baidu and merge them with the original data

In [101]:
baidu = pd.read_csv('data/baidu_2023-11-27.csv') ## this is the data we collected from baidu suggestion using our data collection script
google = pd.read_csv('data/google_2023-11-26.csv') ## this is the data we collected from google suggestion using our data collection script
data = pd.read_csv("data/category_group_data.csv")
target_dict_google = data.groupby('Category')['English'].apply(list).to_dict()
target_dict_baidu = data.groupby('Category')['Translation'].apply(list).to_dict()
# Templates for queries
templates_google = ["why are term_to_use so"]
templates_baidu = ["为什么待替换词汇这么", "待替换词汇为什么这么"]

# DataFrames for suggestions
query_initial_google = pd.DataFrame(columns=['query','suggestion','category','group','completion'])
query_initial_baidu = pd.DataFrame(columns=['query', 'suggestion', 'category', 'group', 'completion'])

# Process for Google using English terms
for category in target_dict_google.keys():
    for template in templates_google:
        for group in target_dict_google[category]:
            query = template.replace("term_to_use", group)
            new_row = {'query': query, 'suggestion': None, 'category': category, 'group': group, 'completion': None}
            query_initial_google  = pd.concat([query_initial_google, pd.DataFrame([new_row])], ignore_index=True)
for category in target_dict_baidu.keys():
    for template in templates_baidu:
        for group in target_dict_baidu[category]:
            query = template.replace("待替换词汇", group)
            new_row = {'query': query, 'suggestion': None, 'category': category, 'group': group, 'completion': None}
            query_initial_baidu = pd.concat([query_initial_baidu, pd.DataFrame([new_row])], ignore_index=True)

In [113]:
# Create a DataFrame of queries that are not responded to by Google and Baidu
unresponded_queries_baidu = pd.DataFrame((set(query_initial_baidu['query']) - set(baidu['query'])), columns=['query'])
unresponded_queries_google = pd.DataFrame(list(set(query_initial_google['query']) - set(google['query'])), columns=['query'])

# merge unresponded queries with the original data
unresponded_queries_baidu = unresponded_queries_baidu.merge(query_initial_baidu, on='query', how='left')
unresponded_queries_google = unresponded_queries_google.merge(query_initial_google, on='query', how='left')

# Concatenate 'unresponded data ' with the original  DataFrame
baidu = pd.concat([baidu,unresponded_queries_baidu], ignore_index=True).fillna('')
google = pd.concat([google,unresponded_queries_google], ignore_index=True).fillna('')



### merge all the data together


In [117]:
# Step 2: Merge and Preprocess Data
baidu = pd.merge(baidu, data, left_on='group', right_on='Translation', how='left')
baidu = baidu[['query', 'suggestion', 'category', 'group', 'English', 'completion']]
baidu = baidu.rename(columns={'English': 'group_English'})

# Step 3: Define Helper Functions
def check_suggestion_starts_with_query(row):
    if row['suggestion'] is None:
        return None
    else:
        return row['suggestion'].lower().startswith(row['query'].lower())

def contains_group(row):
    if row['suggestion'] is None:
        return None
    else:
        return row['group'].lower() in row['suggestion'].lower()

# Step 4: Apply Functions and Process Data
baidu['suggestion_starts_with_query'] = baidu.apply(check_suggestion_starts_with_query, axis=1)
google['suggestion_starts_with_query'] = google.apply(check_suggestion_starts_with_query, axis=1)
baidu['group_in_suggestion'] = baidu.apply(contains_group, axis=1)
google['group_in_suggestion'] = google.apply(contains_group, axis=1)

template_1 = "为什么.+(这么)" 
template_2 = "(.+)为什么这么" 
baidu_template_1 = baidu[baidu['query'].str.contains(re.compile(template_1))]
baidu_template_2 = baidu[baidu['query'].str.contains(re.compile(template_2))]
# Create sets for faster lookup
baidu_template_1_set = set(baidu_template_1['query'])
baidu_template_2_set = set(baidu_template_2['query'])

# Add a new column to the 'baidu' dataframe based on the template matching
baidu['template_match'] = baidu['query'].apply(
    lambda x: 'template 1' if x in baidu_template_1_set 
              else ('template 2' if x in baidu_template_2_set 
                    else None)
)

/var/folders/x1/rg74ttxx7bb0_2z1wxpb2z940000gn/T/ipykernel_1029/2001355718.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  baidu_template_1 = baidu[baidu['query'].str.contains(re.compile(template_1))]
/var/folders/x1/rg74ttxx7bb0_2z1wxpb2z940000gn/T/ipykernel_1029/2001355718.py:28: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  baidu_template_2 = baidu[baidu['query'].str.contains(re.compile(template_2))]


In [105]:
baidu

,query,suggestion,category,group,group_English,completion,suggestion_starts_with_query,group_in_suggestion,template_match
0,为什么儿童这么,为什么儿童这么多哮喘,Age,儿童,children,多哮喘,True,True,template 1
1,为什么儿童这么,为什么儿童这么多白血病,Age,儿童,children,多白血病,True,True,template 1
2,为什么儿童这么,为什么儿童这么多罕见病呢,Age,儿童,children,多罕见病呢,True,True,template 1
3,为什么儿童这么,为什么儿童这么爱出汗,Age,儿童,children,爱出汗,True,True,template 1
4,为什么儿童这么,儿童为什么说话这么晚,Age,儿童,children,儿童为什么说话这么晚,False,True,template 1
...,...,...,...,...,...,...,...,...,...
2350,先生们为什么这么,None,Gender,先生们,gentlemen,None,None,None,template 2
2351,宗教人士为什么这么,None,Religion,宗教人士,religions,None,None,None,template 2
2352,为什么社会主义者这么,None,Political,社会主义者,socialists,None,None,None,template 1
2353,潮人为什么这么,None,Lifestyle,潮人,hipsters,None,None,None,template 2


### baidu translation fucntion 

In [106]:
# Function to generate the MD5 signature required for Baidu Translate API
def make_md5(s, encoding='utf-8'):
    return hashlib.md5(s.encode(encoding)).hexdigest()

# Function to translate text using Baidu Translate API
def translate_baidu(text, app_id, secret_key, from_lang, to_lang):
    base_url = 'https://fanyi-api.baidu.com/api/trans/vip/translate'
    salt = '12345'  # You can use any random number here
    sign = make_md5(app_id + text + salt + secret_key)

    params = {
        'q': text,
        'from': from_lang,
        'to': to_lang,
        'appid': app_id,
        'salt': salt,
        'sign': sign
    }

    response = requests.get(base_url, params=params)
    result = json.loads(response.text)

    # Extracting translation
    if 'trans_result' in result:
        return result['trans_result'][0]['dst']
    else:
        return "Error in translation"

# Your Baidu Translate API credentials
app_id = '123456789' # Your App ID
secret_key = '123456789' # Your Secret Key
# # Translate 'completion' columns
baidu['completion_English'] = baidu['completion'].apply(lambda x: translate_baidu(x, app_id, secret_key, 'zh', 'en'))

KeyboardInterrupt: 

### Extracting sentiment from google and baidu 

In [121]:
#  Filter google data for sentiment analysis 
def extract_after_keyword(suggestion, keyword="so"):
    if suggestion is None:
        return ""
    else:
        pos = suggestion.find(keyword)
        if pos != -1:
            # Return the part of the string including and after the keyword
            return suggestion[pos:].strip()
        return ""  # Return empty string if the keyword is not found]
google.loc[:, 'extracted_suggestion'] = google['suggestion'].apply(lambda x: extract_after_keyword(x))

In [122]:
def extract_after_keyword(suggestion, keyword="这么"):
    if suggestion is None:
        return ""
    else:
        pos = suggestion.find(keyword)
        if pos != -1:
            # Return the part of the string including and after the keyword
            return suggestion[pos:].strip()
        return ""  # Return empty string if the keyword is not found
baidu.loc[:, 'extracted_suggestion'] = baidu['suggestion'].apply(lambda x: extract_after_keyword(x))

### Sentiment Analysis

In [119]:

class SharedCounter:
    shared_count = 0

    @classmethod
    def increment(cls):
        cls.shared_count += 1
        return cls.shared_count

def analyze_sentiment(tweet_text):
    # Function to create a chat message
    def create_chat_message(content, role="user"):
        return {"role": role, "content": content}

    # Setup for using the chat model
    sentiment_analysis_message = (
        "Is the sentiment of this text positive, neutral, or negative?\n"
        "Answer only with a number: 1 if positive, 2 if neutral, and 3 if negative.\n"
        "Here is the text:\n"
        f"{tweet_text}"
    )
    messages = [
        create_chat_message("Act as a careful and accurate zero-shot text classifier", role="system"),
        create_chat_message(sentiment_analysis_message)
    ]

    # Chat API call for sentiment rating
    response = openai.ChatCompletion.create(
        # model="gpt-3.5-turbo-1106",
        model = 'gpt-4-1106-preview',
        messages=messages,
        max_tokens=10,
        request_timeout = 600
    )

    # Extracting response for sentiment rating
    sentiment_rating = response.choices[0].message['content'].strip()

    # Ensure the rating is a number and within the expected range
    if sentiment_rating.isdigit() and sentiment_rating in ["1", "2", "3"]:
        sentiment_rating = int(sentiment_rating)
    else:
        sentiment_rating = "Invalid response"

    # Result
    result = {
        "sentiment_rating": sentiment_rating
    }

    return result

In [123]:
### Sentiment Analysis for completion part 
baidu['sentiment_rating_completion'] = baidu[baidu['suggestion_starts_with_query']]['extracted_suggestion'].apply(lambda x: analyze_sentiment(x)['sentiment_rating'])
google['sentiment_rating_completion'] = google[google['suggestion_starts_with_query']]['extracted_suggestion'].apply(lambda x: analyze_sentiment(x)['sentiment_rating'])

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
### Sentiment Analysis for full suggestion
baidu['sentiment_rating_full_suggestion'] = baidu[baidu['suggestion_starts_with_query']]['suggestion'].apply(lambda x: analyze_sentiment(x)['sentiment_rating'])
google['sentiment_rating_full_suggestion'] = google[google['suggestion_starts_with_query']]['suggestion'].apply(lambda x: analyze_sentiment(x)['sentiment_rating'])

In [ ]:
google.to_csv('data/google.csv',index=False)
baidu.to_csv('data/baidu.csv',index=False)

In [ ]:

# ## if you find there are some inconsisent sentiment rating, you can use the following function to assign the most common sentiment rating to the suggestion. 
# # Function to assign common sentiment rating
# def assign_common_sentiment_rating(df, rating_col, new_col):
#     common_ratings = df.groupby('extracted_suggestion')[rating_col].agg(lambda x: pd.Series.mode(x)[0])
#     df[new_col] = df['extracted_suggestion'].map(common_ratings)
#     df[new_col] = pd.to_numeric(df[new_col], errors='coerce')
#     return df
